In [1]:
import tensorflow as tf
print(tf.__version__)


2.13.1


In [3]:
import tfx
print(tfx.__version__)


1.14.0


In [5]:
# !pip install --upgrade tfx tfx-bsl


# DATA INGESTION

In [4]:
import os
from tfx.components import ImportExampleGen , CsvExampleGen
from tfx.orchestration import metadata
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [4]:
base_dir = os.getcwd()
print("base directory:",base_dir)
Csv_data_dir = "../data"
print(" csv file data direcrory",Csv_data_dir) 
example_gen = CsvExampleGen(input_base=os.path.join(base_dir, Csv_data_dir)) # use only if my data in data dirctory 

# example_gen = ImportExampleGen(input_base=(os.path.join(base_dir, data_dir))) # the ImportExampleGen component in TFX is designed to import data in TFRecord format, not CSV format. so we can't use this method
# # however if we wish to use custom data other than csv we can prepare the data and ingest in form of Tfrecord.


base directory: C:\Users\ASUS\building-machine-learning-pipelines\Untitled Folder
 csv file data direcrory ../data


The `pipeline_root` argument specifies the root directory for storing pipeline `artifacts (temporary files, models, etc.)`. Here, it uses '../pipeline_root_learning/tfx2'.
../ refers to the parent directory of the current working directory. So, the pipeline root will be located two directories above the current one, in a subdirectory named tfx2 within a directory called pipeline_root_learning

In [5]:
# Create an InteractiveContext to execute the component
context = InteractiveContext(pipeline_root='../pipeline_root_learning/tfx2')   # ".." means the parent directory
context.run(example_gen) #  triggering the execution of example_gen within the InteractiveContext

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 3
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## In the script to_TFRecord.py we have converted csv into tfrecord , lets utilize it..

In [6]:
base_dir = os.getcwd()
Tf_record_data_dir = os.path.join(os.pardir, "tfrecord_data")

In [7]:
print(base_dir)
print(Tf_record_data_dir) # one level up(means one directory behind) i.e building-machine-lerarning-pipeline folder

C:\Users\ASUS\building-machine-learning-pipelines\Untitled Folder
..\tfrecord_data


In [8]:
tf_record_location = os.path.join(base_dir, Tf_record_data_dir)
tf_record_location

'C:\\Users\\ASUS\\building-machine-learning-pipelines\\Untitled Folder\\..\\tfrecord_data'

In [9]:
example_gen = ImportExampleGen(input_base=tf_record_location)#ImportExampleGen because we are impoting Tfrecord file format

In [10]:
# Create an InteractiveContext to execute the component
context = InteractiveContext(pipeline_root='../pipeline_root_learning/tfx_for_tfrecordData')   # ".." means the parent directory
context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 248
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Data Preparation

### Splitting one dataset into subsets



-we can extend our data ingestion by requiring a
three-way split: training, evaluation, and test sets with a ratio of 6:2:2. The ratio settings
are defined through the hash_buckets:

In [11]:
from tfx.components import CsvExampleGen
from tfx.proto import example_gen_pb2


In [12]:
# defining the output configurations

output = example_gen_pb2.Output(
split_config=example_gen_pb2.SplitConfig(splits=[
example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=6),
example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2),
example_gen_pb2.SplitConfig.Split(name='test', hash_buckets=2)]))
print(output)

split_config {
  splits {
    name: "train"
    hash_buckets: 6
  }
  splits {
    name: "eval"
    hash_buckets: 2
  }
  splits {
    name: "test"
    hash_buckets: 2
  }
}



In [13]:
os.listdir("../tfrecord_data")

['.ipynb_checkpoints', 'housing.tfrecord']

In [14]:
example_gen = ImportExampleGen(input_base=os.path.join(base_dir,"../tfrecord_data") , output_config=output ) # use only if my data in data dirctory 

# # example_gen = CsvExampleGen(input_base=os.path.join(base_dir,"data"), output_config=output)


In [15]:
context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 249
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [39]:
# for artifact in example_gen.outputs['examples'].get():
#     print(artifact)

standalone setup of data validation is in Datavalidation.ipynb

## Integrating TFDV into Your Machine Learning Pipeline

So far, all methods we have discussed can be used in a standalone setup. This can be
helpful to investigate datasets outside of the pipeline setup.
TFX provides a pipeline component called StatisticsGen, which accepts the output
of the previous ExampleGen components as input and then performs the generation
of statistics:

In [100]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 40
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [101]:
# we can visualize the output in an interactive context using:
context.show(statistics_gen.outputs['statistics'])

## Schema

In [102]:
from tfx.components import SchemaGen 
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 41
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

With the statistics and schema in place, we can now validate our new dataset

In [103]:
from tfx.components import ExampleValidator
example_validator = ExampleValidator(
statistics=statistics_gen.outputs['statistics'],
schema=schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 42
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

If the
component detects a misalignment in the dataset statistics
ExampleValidator
or schema between the new and the previous dataset, it will set the status to failed in
the metadata store, and the pipeline ultimately stops. Otherwise, the pipeline moves
on to the next step, the data preprocessing.